In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))


In [ ]:
from telebot.types import InlineKeyboardMarkup, InlineKeyboardButton
from ipynb.fs.full.chamar import init
from ipynb.fs.full.dalbem import inita
%run alex.ipynb
import os
import string
import time
import shutil
import telebot
import requests
from pdf2image import convert_from_path


with open ('C:/Users/pdv/Desktop/gabriel/chaveapitelebot.txt', 'r') as chaves:
    for chave in chaves:
        chaveapi = chave

        
bot = telebot.TeleBot(chaveapi)
cnpjs,ljs = [],[]
mensagemID,forn = '',''
mes_global = '01'
lista_lojas = ['LJ1 - Guanabara','LJ2 - Primavera','LJ3 - Iguatemi','LJ4 - Flamboyant',
                   'LJ5 - São Paulo','LJ6 - Valinhos','LJ7 - Ouro Verde', 'ESCOLHIDO']


def makeKeyboard(lista):
    markup = InlineKeyboardMarkup()
    if len(lista) == 8:
        for i in range(len(lista)):
            markup.add(InlineKeyboardButton(str(lista[i]), callback_data=str(lista[i])))
    else:
        for i in range(len(lista)):
            markup.add(InlineKeyboardButton(str(lista[i]), callback_data=str(i)))
    return markup


def lojas():
    global lista_lojas,mensagemID
    bot.send_message(mensagemID,"Escolha quais lojas: ", reply_markup = makeKeyboard(lista_lojas))
    
    
def checkforn(forn = 'adep',ljs='0123456'):
    global cnpjs,mensagemID, mes_global
    perg,achei,aux,cnpj,cnpjs,k = False,False,[],'',[],0
    with open("cnpjs.txt", 'r') as db:
        for linha in db:
            if forn in linha.lower():
                cnpj = linha[-19:-9]
                cnpj = ''.join([i for i in cnpj if i not in string.punctuation])
                cnpjs.append(int(cnpj))
                aux.append(linha)
                k+=1
                if achei:
                    perg = True
                achei = True
    if not achei:
        bot.send_message(mensagemID, 'não achei esse fornecedor')
        return 0
    if perg:
        if len(aux) == 8:
            aux.append('fornecedor fantasma pra nao atrapalhar o sistema')
        bot.send_message(mensagemID,"Escolha seu fornecedor: ", reply_markup = makeKeyboard(aux))
        return 1
    if achei and perg == False:
        try:
            global mes_global
            init(int(cnpj),loja=ljs,idz=mensagemID,mes=mes_global)
            envia(mensagemID)
        except Exception as e:
            print(e)
            bot.send_message(mensagemID, 'não encontrei notas. desculpe, error :',e)


@bot.message_handler(commands=["notas"])
def notas(mensagem):
    global forn,mensagemID,ljs
    ljs=[]
    mensagemID = mensagem.chat.id
    forn = mensagem.text[7:]        
    lojas()
    

@bot.message_handler(commands=["oba"])
def oba(mensagem):
#     url = 'https://obahortifruti.com.br/ofertas/minha-hora-oba/index.php?regiao=sp'
#     response = requests.get(url, stream=True)
#     with open('C:/Users/pdv/Desktop/selenium-imgs/interior.png', 'wb') as out_file:
#         shutil.copyfileobj(response.raw, out_file)
#     del response
    pasta = 'C:/Users/pdv/Desktop/selenium-imgs/oba'
    mensagemID = mensagem.chat.id
    for diretorio, subpastas, arquivos in os.walk(pasta):
        for arquivo in arquivos:
            pic = f'{pasta}\\{arquivo}'
            bot.send_photo(mensagemID, photo=open(pic,'rb'))
            
            
@bot.message_handler(commands=["goodbom"])
def goodbom(mensagem):
#     url = 'https://goodbom.com.br/tabloides/index.php/goodbom-taquaral/'
    pasta = 'C:/Users/pdv/Desktop/selenium-imgs/goodbom'
    mensagemID = mensagem.chat.id
    for diretorio, subpastas, arquivos in os.walk(pasta):
        for arquivo in arquivos:
            pic = f'{pasta}\\{arquivo}'
            bot.send_photo(mensagemID, photo=open(pic,'rb'))
            
    
@bot.message_handler(commands=["fartura"])
def fartura(mensagem):
#     url = 'https://www.hortifrutifartura.com.br/wp-content/themes/fartura/images/banner-oferta.png'
#     response = requests.get(url, stream=True)
#     with open('C:/Users/pdv/Desktop/selenium-imgs/nosso.png', 'wb') as out_file:
#         shutil.copyfileobj(response.raw, out_file)
#     del response
    mensagemID = mensagem.chat.id
    pic = 'C:/Users/pdv/Desktop/selenium-imgs/nosso.png'
    bot.send_photo(mensagemID, photo=open(pic,'rb'))


@bot.message_handler(commands=["obasp"])
def obasp(mensagem):
    url = 'https://obahortifruti.com.br/ofertas/minha-hora-oba/index.php?regiao=sp'
    response = requests.get(url, stream=True)
    with open('C:/Users/pdv/Desktop/selenium-imgs/sp.png', 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)
    del response
    mensagemID = mensagem.chat.id
    pic = 'C:/Users/pdv/Desktop/selenium-imgs/sp.png'
    bot.send_photo(mensagemID, photo=open(pic,'rb'))
    
    
@bot.message_handler(commands=["dalbem"])
def dalbem(mensagem):
    mensagemID = mensagem.chat.id
#     inita()
    enviadalbem(mensagemID)
    

@bot.message_handler(commands=["pgmenos"])
def pgmenos(mensagem):
    pasta = 'C:/Users/pdv/Desktop/selenium-imgs/pgmenos/feito'
    mensagemID = mensagem.chat.id
    for diretorio, subpastas, arquivos in os.walk(pasta):
        for arquivo in arquivos:
            pic = f'{pasta}\\{arquivo}'
            bot.send_photo(mensagemID, photo=open(pic,'rb'))
            
            
@bot.message_handler(commands=["alex"])
def alex(mensagem):
    mensagemID = mensagem.chat.id
    dados = lecao()
    for i in dados:
        bot.send_message(mensagemID,i)

   


@bot.message_handler(commands=["numero"])
def numero(mensagem):
    msg = str(int(mensagem.text[8:]))
    global forn,mensagemID,ljs, mes_global
    ljs=[]
    mensagemID = mensagem.chat.id
    if len(msg) >= 3 and msg.isdigit() == True:
        print(f'por cnpj: {msg}')
        init(numero=msg,idz=mensagemID,mes=mes_global)
        envia(mensagemID)
    else:
        bot.send_message(mensagemID,"Não use letras ou menos de 3 números")

    
@bot.message_handler(func=lambda message: True)
def resposta(mensagem):
    text = '''Olá! Seja bem vindo ao NF Fartura Bot
    ^^  Estes são os meus Comandos:

    PARA PUXAR NOTAS PELO NOME DO FORNECEDOR:
    /notas fornecedor (ex: /notas adep)
    /notas cnpj (ex: /notas 49.092.398/0003-94 ou /notas 49.092)
    
    PARA PUXAR NOTAS PELO NUMERO DE NOTA:
    /numero n°NF (ex: /numero 15945)
    
    PARA PUXAR OFERTAS DO DALBEM: /dalbem
    
    PARA PUXAR OFERTAS DO OBA: /oba
    
    PARA PUXAR OFERTAS FARTURA: /fartura
    =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-='''
    bot.reply_to(mensagem, text)
    
    
@bot.callback_query_handler(func=lambda call: True)
def handle_query(call):
    global lista_lojas,ljs,forn
    if call.data.isdigit():
        try:
            iniciar(call.data)
        except Exception as e:
            bot.send_message(mensagemID, 'não encontrei notas. desculpe, erro ',e)
    else:
        if call.data == 'ESCOLHIDO' and ljs:
            checkforn(forn,ljs)
        if str(lista_lojas.index(call.data)) not in ljs and call.data != 'ESCOLHIDO':
            ljs.append(str(lista_lojas.index(call.data)))
        
        
def envia(mensagemID):
    k=0
    pasta = fr'C:\Users\pdv\projetos_jupyter\retorno\\{mensagemID}'
    for diretorio, subpastas, arquivos in os.walk(pasta):
        if not arquivos:
            text = 'Desculpe, não achei notas desse fornecedor nessa loja'
            bot.send_message(mensagemID,text)
            return 0
        for arquivo in arquivos:
            j=str(k)+'.'
            x = f'{pasta}\\nota{j}png'
            k+= 1
            bot.send_photo(mensagemID, photo=open(x,'rb'))
    text = 'pronto :)'
    bot.send_message(mensagemID,text)
    
    
def enviadalbem(mensagemID):
    pasta = r'C:\Users\pdv\Desktop\selenium-imgs\rename'
    for diretorio, subpastas, arquivos in os.walk(pasta):
        if not arquivos:
            text = 'Desculpe, não achei as ofertas do dalbem'
            bot.send_message(mensagemID,text)
            return 0
        for arquivo in arquivos:
            x = f'{pasta}\\{arquivo}'
            bot.send_photo(mensagemID, photo=open(x,'rb'))
    text = 'pronto :)'
    bot.send_message(mensagemID,text)
    
    
def iniciar(calldata):
    global cnpjs,mensagemID,ljs, mes_global
    try:
        r = init(cnpjs[int(calldata)],loja=ljs,idz=mensagemID,mes=mes_global)
        envia(mensagemID)
    except Exception as e:
        print(e)
        bot.reply_to(mensagemID, 'não achei notas')
    if r == "não achei esse":
        bot.reply_to(mensagemID, r)

        
while True:
    try:
        bot.polling(none_stop=True, interval=0, timeout=0)
    except:
        time.sleep(10)

['m']
['m']
['m']
['m']
['=-=-=-=-=-=-=açougue:=-=-=-=-=-=-=', 'loja 2:', '1° dezena:  0.09 % de crescimento', '2° dezena:  0.03 % de crescimento', '3° dezena:  -0.37 % de crescimento', '=-=-=-=-=-=-=frios:=-=-=-=-=-=-=', 'loja 2:', '1° dezena:  -0.11 % de crescimento', '2° dezena:  -0.06 % de crescimento', '3° dezena:  -0.53 % de crescimento', '=-=-=-=-=-=-=auto serviço:=-=-=-=-=-=-=', 'loja 2:', '1° dezena:  -0.05 % de crescimento', '2° dezena:  0.1 % de crescimento', '3° dezena:  -0.32 % de crescimento', '=-=-=-=-=-=-=mercearia:=-=-=-=-=-=-=', 'loja 2:', '1° dezena:  -0.12 % de crescimento', '2° dezena:  -0.02 % de crescimento', '3° dezena:  -0.49 % de crescimento']
['m']
['m']
['m']
['m']
['m']
['m']
['m']
['m']
por cnpj: 894799
['m']
['m']
['m']
['m']
['m']
['m']
['m']
['m']


In [3]:
%run alex.ipynb
print(lecao)

<Worksheet "açougue">
<Worksheet "frios">
<Worksheet "auto">
<Worksheet "mercearia">
mercearia: loja 1: 1° dezena 01 - 10:  -0.25% de crescimento 2° dezena 11 - 20:  -0.12% de crescimento 3° dezena 21 - 31:  -0.53% de crescimento loja 2: 1° dezena 01 - 10:  -0.12% de crescimento 2° dezena 11 - 20:  -0.02% de crescimento 3° dezena 21 - 31:  -0.49% de crescimento loja 3: 1° dezena 01 - 10:  -0.21% de crescimento 2° dezena 11 - 20:  -0.04% de crescimento 3° dezena 21 - 31:  -0.51% de crescimento loja 4: 1° dezena 01 - 10:  -0.23% de crescimento 2° dezena 11 - 20:  -0.13% de crescimento 3° dezena 21 - 31:  -0.58% de crescimento loja 5: 1° dezena 01 - 10:  -0.11% de crescimento 2° dezena 11 - 20:  -0.14% de crescimento 3° dezena 21 - 31:  -0.55% de crescimento loja 6: 1° dezena 01 - 10:  -0.14% de crescimento 2° dezena 11 - 20:  -0.13% de crescimento 3° dezena 21 - 31:  -0.61% de crescimento loja 7: 1° dezena 01 - 10:  -0.06% de crescimento 2° dezena 11 - 20:  -0.08% de crescimento 3° dezen

In [2]:
from pdf2image import convert_from_path
for i in range(2):
    images = convert_from_path(f'C:/Users/pdv/Desktop/selenium-imgs/pgmenos/oferta{i}.pdf',
                               poppler_path=r'C:\Program Files (x86)\poppler-21.11.0\Library\bin',
                              output_folder='C:/Users/pdv/Desktop/selenium-imgs/pgmenos/feito',
                              fmt='PNG')
    for i in range(len(images)):   
        images[i].save('page'+ str(i) +'.png', 'PNG')